# <center>Learning how to play Space Invaders with RL.</center>

![space invaders poster image](https://cdn-images-1.medium.com/max/1200/1*Ya6drllhmC82yh0J-elrPA.jpeg)

In [1]:
import gym
env = gym.make('SpaceInvaders-v0')

**Observation space**: the observation space consists on 3 sequential images of size 210x160

**Action space**: there are 6 possible actions: *left, right, shoot, left-shoot, right-shoot, do nothing*.

In [2]:
print(env.observation_space.shape)
print(env.action_space)

(210, 160, 3)
Discrete(6)


In [3]:
# Example of how env works
total_reward = 0
env.reset()
while True:
    env.render()
    next_state, reward, done, info = env.step((env.action_space.sample())) # select random action
    total_reward += reward
    if done:
        print(total_reward)
        break
        
env.close()

210.0


## Training

In [7]:
from dqn.dqnAgent import DQNAgent
from dqn.model import Model
import numpy as np

agent = DQNAgent(env)
episodes = 1000
rewards = []

for e in range(1, episodes):
    # reset state at the beggining of each game
    state = env.reset()
    Model.img_preprocessing(state)

    state = Model.img_preprocessing(state)

    total_reward = 0
    
    while(True):
        env.render()
        
        action = agent.act(state)
        next_state, reward, done, info = env.step(action)
        next_state = Model.img_preprocessing(next_state)
        
        agent.memory.add(state, action, reward, next_state, done)
        
        state = next_state
        
        total_reward += reward
        
        if done:
            print("episode: {}, score: {}".format(e, total_reward))
            rewards.append(total_reward)
            break
      
    env.close()
    # train the agent with the experience of the episode
    agent.learn(32)
    

episode: 1, score: 65.0
episode: 2, score: 100.0
episode: 3, score: 55.0
episode: 4, score: 425.0


KeyboardInterrupt: 